# Power Grid Stress Prediction using Recurrent Neural Networks
## Implementing and Comparing Different RNN Architectures for Time Series Data
### Pablo X Zumba

This is classification task using RNNs (i.e., a sequence to value prediction). We have hourly power consumption of households for 12 hours. Based on this, we will determine whether the power grid is strained (1) or not (0). 

Therefore, use the columns from `Hour 0` to `Hour 11` to predict the `target` column in the `power.csv` data set.

Don't forget to adjust the number of neurons in the input layers correctly. Otherwise, there could be errors.

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error


# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)



2023-02-07 11:29:17.003030: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Read the Dataset

In [2]:
power = pd.read_csv('power.csv')

power.head()

,Hour 0,Hour 1,Hour 2,Hour 3,Hour 4,Hour 5,Hour 6,Hour 7,Hour 8,Hour 9,Hour 10,Hour 11,target
0,2.550633,2.523400,2.582333,2.541667,2.475733,2.476233,2.455800,2.447200,2.441733,3.146133,2.661733,2.576000,1
1,1.596933,1.619567,2.473733,2.731133,2.431133,2.479667,1.690200,1.332133,1.375167,1.050900,0.585900,2.651900,1
2,0.534933,0.540467,0.575367,0.526500,0.521900,0.565333,1.426467,0.602067,0.547433,0.525067,1.270300,0.393767,0
3,1.085867,0.651233,0.634600,0.653000,0.646067,0.628400,0.611067,0.612533,0.660100,0.606067,1.471867,0.834533,0
4,0.456000,0.286300,0.310833,0.250933,0.277667,0.308633,0.610400,1.563533,1.421867,3.324400,3.207567,1.425433,1


In [3]:
power.shape

(1417, 13)

# Split the Data



In [4]:
# First 1000 days are for train
train = power.iloc[:1000]

# Remaining 417 days are for test
test = power.iloc[-417:]

In [5]:
train.shape

(1000, 13)

In [6]:
test.shape

(417, 13)

# Create Input and Target values

The first 12 columns (hourly data) will be input to predict the last column (i.e., target)

In [7]:
# The first 12 columns (from 0 to 11) are inputs

train_inputs = train.iloc[:,:12]

In [8]:
train_inputs.shape

(1000, 12)

## Add one more dimension to make it ready for RNNs

In [9]:
#Create an additional dimension for train

train_x = np.array(train_inputs).reshape(1000,12,1)

train_x

array([[[2.55063333],
        [2.5234    ],
        [2.58233333],
        ...,
        [3.14613333],
        [2.66173333],
        [2.576     ]],

       [[1.59693333],
        [1.61956667],
        [2.47373333],
        ...,
        [1.0509    ],
        [0.5859    ],
        [2.6519    ]],

       [[0.53493333],
        [0.54046667],
        [0.57536667],
        ...,
        [0.52506667],
        [1.2703    ],
        [0.39376667]],

       ...,

       [[1.5426    ],
        [0.79336667],
        [0.70356667],
        ...,
        [1.8318    ],
        [1.2689    ],
        [0.35363333]],

       [[0.4064    ],
        [0.7657    ],
        [0.3216    ],
        ...,
        [0.8938    ],
        [0.7913    ],
        [0.86173333]],

       [[0.32603333],
        [0.3344    ],
        [0.28883333],
        ...,
        [1.36266667],
        [1.54143333],
        [1.4652    ]]])

## Set the target

In [10]:
# The last column is TARGET

train_target = train.iloc[:,-1]

In [11]:
train_target.shape

(1000,)

## Repeat for TEST

In [12]:
test.shape

(417, 13)

In [13]:
# The first 12 columns are inputs

test_inputs = test.iloc[:,:12]

In [14]:
#Create an additional dimension for test

test_x = np.array(test_inputs).reshape(417,12,1)

test_x.shape 

(417, 12, 1)

In [15]:
# The last column is TARGET

test_target = test.iloc[:,-1]

In [16]:
test_target.shape

(417,)

# Baseline Accuracy

In [17]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.505


In [20]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.49640287769784175


# Build a normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [21]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[12, 1]),
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

2023-02-07 11:29:22.805694: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_target, epochs=50,
                    validation_data=(test_x, test_target))

Epoch 1/50
32/32 [==============================] - 1s 11ms/step - loss: 0.6368 - accuracy: 0.6360 - val_loss: 0.6373 - val_accuracy: 0.6043
Epoch 2/50
32/32 [==============================] - 0s 3ms/step - loss: 0.5718 - accuracy: 0.7110 - val_loss: 0.5791 - val_accuracy: 0.7098
Epoch 3/50
32/32 [==============================] - 0s 3ms/step - loss: 0.5249 - accuracy: 0.7380 - val_loss: 0.6450 - val_accuracy: 0.6139
Epoch 4/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4932 - accuracy: 0.7520 - val_loss: 0.5273 - val_accuracy: 0.7026
Epoch 5/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4749 - accuracy: 0.7600 - val_loss: 0.5317 - val_accuracy: 0.7026
Epoch 6/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4668 - accuracy: 0.7460 - val_loss: 0.5159 - val_accuracy: 0.6882
Epoch 7/50
32/32 [==============================] - 0s 3ms/step - loss: 0.4644 - accuracy: 0.7520 - val_loss: 0.5318 - val_accuracy: 0.7434
Epoch 8/50
32/32 [=

In [23]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5163407921791077, 0.7362110018730164]

In [24]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.52
accuracy: 73.62%


# Build a simple RNN with one layer

In [25]:
n_steps = 12
n_inputs = 1


model = keras.models.Sequential([
    
    keras.layers.SimpleRNN(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [26]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [27]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_target, epochs=50,
                    validation_data=(test_x, test_target), callbacks=callback)

Epoch 1/50
32/32 [==============================] - 1s 13ms/step - loss: 0.5760 - accuracy: 0.7020 - val_loss: 0.5558 - val_accuracy: 0.7242
Epoch 2/50
32/32 [==============================] - 0s 5ms/step - loss: 0.5172 - accuracy: 0.7310 - val_loss: 0.5855 - val_accuracy: 0.6930
Epoch 3/50
32/32 [==============================] - 0s 5ms/step - loss: 0.4897 - accuracy: 0.7510 - val_loss: 0.9441 - val_accuracy: 0.5635
Epoch 4/50
32/32 [==============================] - 0s 5ms/step - loss: 0.5221 - accuracy: 0.7310 - val_loss: 0.5472 - val_accuracy: 0.7098
Epoch 5/50
32/32 [==============================] - 0s 6ms/step - loss: 0.4806 - accuracy: 0.7640 - val_loss: 0.5458 - val_accuracy: 0.7002
Epoch 6/50
32/32 [==============================] - 0s 6ms/step - loss: 0.4693 - accuracy: 0.7620 - val_loss: 0.5159 - val_accuracy: 0.7170
Epoch 7/50
32/32 [==============================] - 0s 6ms/step - loss: 0.4693 - accuracy: 0.7570 - val_loss: 0.6534 - val_accuracy: 0.6475
Epoch 8/50
32/32 [=

In [28]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5172591805458069, 0.7577937841415405]

In [29]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.52
accuracy: 75.78%


In [30]:
# Predictions are probabilities.

predictions = model.predict(test_x)

14/14 [==============================] - 0s 2ms/step


In [31]:
# Rounding the probabilities determines 1 or 0

np.round(predictions)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],

In [32]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_target, np.round(predictions))

array([[186,  24],
       [ 77, 130]])

# Build a simple RNN with two or more layers (Deep RNN)

**Be careful: when stacking RNN layers, you have to set "return_sequences" to True. This enables the layer to send a "sequence" of values to the next layer -- jut like how it uses a sequence of values for training.**

**Since the last layer is DENSE, it can't take sequence data. Therefore, you CANNOT return sequences from the previous layer. So, remove** `return_sequences` **from previous layer.**

In [33]:
n_steps = 12
n_inputs = 1


model = keras.models.Sequential([
    keras.layers.SimpleRNN(24, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(24, return_sequences=True),
    keras.layers.SimpleRNN(24), 
    keras.layers.Dense(1, activation='sigmoid')
])

In [34]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 3s 22ms/step - loss: 0.5932 - accuracy: 0.6840 - val_loss: 0.7807 - val_accuracy: 0.6139
Epoch 2/20
32/32 [==============================] - 0s 9ms/step - loss: 0.5571 - accuracy: 0.7180 - val_loss: 0.5610 - val_accuracy: 0.6978
Epoch 3/20
32/32 [==============================] - 0s 9ms/step - loss: 0.5304 - accuracy: 0.7380 - val_loss: 0.9165 - val_accuracy: 0.5588
Epoch 4/20
32/32 [==============================] - 0s 9ms/step - loss: 0.5420 - accuracy: 0.7310 - val_loss: 0.5466 - val_accuracy: 0.6978
Epoch 5/20
32/32 [==============================] - 0s 9ms/step - loss: 0.5027 - accuracy: 0.7520 - val_loss: 0.5888 - val_accuracy: 0.6954
Epoch 6/20
32/32 [==============================] - 0s 9ms/step - loss: 0.4869 - accuracy: 0.7640 - val_loss: 0.5129 - val_accuracy: 0.7122
Epoch 7/20
32/32 [==============================] - 0s 9ms/step - loss: 0.4869 - accuracy: 0.7480 - val_loss: 0.5734 - val_accuracy: 0.7338
Epoch 8/20
32/32 [=

In [35]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5364943742752075, 0.7074340581893921]

In [36]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.54
accuracy: 70.74%


# Build a LSTM with one layer

In [37]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(24, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [38]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 3s 34ms/step - loss: 0.5881 - accuracy: 0.6850 - val_loss: 0.5543 - val_accuracy: 0.7290
Epoch 2/20
32/32 [==============================] - 0s 8ms/step - loss: 0.5104 - accuracy: 0.7420 - val_loss: 0.6199 - val_accuracy: 0.6811
Epoch 3/20
32/32 [==============================] - 0s 7ms/step - loss: 0.4863 - accuracy: 0.7440 - val_loss: 0.9323 - val_accuracy: 0.5683
Epoch 4/20
32/32 [==============================] - 0s 8ms/step - loss: 0.4834 - accuracy: 0.7530 - val_loss: 0.5367 - val_accuracy: 0.7098
Epoch 5/20
32/32 [==============================] - 0s 11ms/step - loss: 0.4718 - accuracy: 0.7570 - val_loss: 0.5368 - val_accuracy: 0.7026
Epoch 6/20
32/32 [==============================] - 0s 7ms/step - loss: 0.4642 - accuracy: 0.7590 - val_loss: 0.5177 - val_accuracy: 0.7098
Epoch 7/20
32/32 [==============================] - 0s 6ms/step - loss: 0.4672 - accuracy: 0.7630 - val_loss: 0.5215 - val_accuracy: 0.7530
Epoch 8/20
32/32 [

In [39]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5298769474029541, 0.7194244861602783]

In [40]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.53
accuracy: 71.94%


# Build a LSTM with two or more layers

In [41]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [42]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 6s 50ms/step - loss: 0.6202 - accuracy: 0.6620 - val_loss: 0.7729 - val_accuracy: 0.5683
Epoch 2/20
32/32 [==============================] - 0s 16ms/step - loss: 0.5618 - accuracy: 0.7110 - val_loss: 0.6823 - val_accuracy: 0.6499
Epoch 3/20
32/32 [==============================] - 1s 17ms/step - loss: 0.5256 - accuracy: 0.7470 - val_loss: 1.0197 - val_accuracy: 0.5204
Epoch 4/20
32/32 [==============================] - 0s 16ms/step - loss: 0.5424 - accuracy: 0.7320 - val_loss: 0.5767 - val_accuracy: 0.7050
Epoch 5/20
32/32 [==============================] - 1s 16ms/step - loss: 0.4857 - accuracy: 0.7600 - val_loss: 0.5243 - val_accuracy: 0.7074
Epoch 6/20
32/32 [==============================] - 1s 16ms/step - loss: 0.4658 - accuracy: 0.7570 - val_loss: 0.5120 - val_accuracy: 0.7098
Epoch 7/20
32/32 [==============================] - 0s 16ms/step - loss: 0.4704 - accuracy: 0.7490 - val_loss: 0.5589 - val_accuracy: 0.7698
Epoch 8/20
32

In [43]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5327463746070862, 0.7410072088241577]

In [44]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.53
accuracy: 74.10%


# Build a GRU with one layer

In [45]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(24, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='relu')
])

In [46]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 3s 20ms/step - loss: 7.7896 - accuracy: 0.4950 - val_loss: 7.6570 - val_accuracy: 0.5036
Epoch 2/20
32/32 [==============================] - 0s 7ms/step - loss: 7.7896 - accuracy: 0.4950 - val_loss: 7.6570 - val_accuracy: 0.5036
Epoch 3/20
32/32 [==============================] - 0s 7ms/step - loss: 7.7896 - accuracy: 0.4950 - val_loss: 7.6570 - val_accuracy: 0.5036
Epoch 4/20
32/32 [==============================] - 0s 7ms/step - loss: 7.7896 - accuracy: 0.4950 - val_loss: 7.6570 - val_accuracy: 0.5036
Epoch 5/20
32/32 [==============================] - 0s 7ms/step - loss: 7.7896 - accuracy: 0.4950 - val_loss: 7.6570 - val_accuracy: 0.5036
Epoch 6/20
32/32 [==============================] - 0s 8ms/step - loss: 7.7896 - accuracy: 0.4950 - val_loss: 7.6570 - val_accuracy: 0.5036
Epoch 6: early stopping


In [47]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[7.656989097595215, 0.5035971403121948]

In [48]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 7.66
accuracy: 50.36%


# Build a GRU with two or more layers

In [49]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(6, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(6, return_sequences=True),
    keras.layers.GRU(6),
    keras.layers.Dense(1, activation='sigmoid')
])

In [50]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 6s 46ms/step - loss: 0.6179 - accuracy: 0.6640 - val_loss: 0.5684 - val_accuracy: 0.7170
Epoch 2/20
32/32 [==============================] - 0s 15ms/step - loss: 0.5254 - accuracy: 0.7340 - val_loss: 0.6260 - val_accuracy: 0.6811
Epoch 3/20
32/32 [==============================] - 0s 14ms/step - loss: 0.5045 - accuracy: 0.7420 - val_loss: 0.8415 - val_accuracy: 0.5564
Epoch 4/20
32/32 [==============================] - 0s 15ms/step - loss: 0.5058 - accuracy: 0.7400 - val_loss: 0.5441 - val_accuracy: 0.7074
Epoch 5/20
32/32 [==============================] - 0s 14ms/step - loss: 0.4817 - accuracy: 0.7500 - val_loss: 0.5931 - val_accuracy: 0.7074
Epoch 6/20
32/32 [==============================] - 0s 15ms/step - loss: 0.4750 - accuracy: 0.7540 - val_loss: 0.5182 - val_accuracy: 0.7098
Epoch 7/20
32/32 [==============================] - 0s 15ms/step - loss: 0.4711 - accuracy: 0.7520 - val_loss: 0.5245 - val_accuracy: 0.7578
Epoch 8/20
32

In [51]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5011864304542542, 0.7721822261810303]

In [52]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.50
accuracy: 77.22%


# Conv1D

In [53]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=10, kernel_size=3, strides=2, padding="valid", input_shape=[n_steps, n_inputs]),
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=1, padding="valid", dilation_rate=2),
    keras.layers.LSTM(6, return_sequences=True),
    keras.layers.LSTM(6),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 5, 10)             40        
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 20)             620       
                                                                 
 lstm_4 (LSTM)               (None, 1, 6)              648       
                                                                 
 lstm_5 (LSTM)               (None, 6)                 312       
                                                                 
 dense_8 (Dense)             (None, 1)                 7         
                                                                 
Total params: 1,627
Trainable params: 1,627
Non-trainable params: 0
_________________________________________________________________


In [54]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 5s 28ms/step - loss: 0.6677 - accuracy: 0.6000 - val_loss: 0.6613 - val_accuracy: 0.5923
Epoch 2/20
32/32 [==============================] - 0s 6ms/step - loss: 0.5830 - accuracy: 0.7170 - val_loss: 0.6280 - val_accuracy: 0.6715
Epoch 3/20
32/32 [==============================] - 0s 5ms/step - loss: 0.5547 - accuracy: 0.7310 - val_loss: 0.8472 - val_accuracy: 0.5516
Epoch 4/20
32/32 [==============================] - 0s 6ms/step - loss: 0.5384 - accuracy: 0.7410 - val_loss: 0.5796 - val_accuracy: 0.7122
Epoch 5/20
32/32 [==============================] - 0s 6ms/step - loss: 0.5296 - accuracy: 0.7570 - val_loss: 0.5830 - val_accuracy: 0.7002
Epoch 6/20
32/32 [==============================] - 0s 5ms/step - loss: 0.5228 - accuracy: 0.7410 - val_loss: 0.5534 - val_accuracy: 0.7002
Epoch 7/20
32/32 [==============================] - 0s 5ms/step - loss: 0.5227 - accuracy: 0.7350 - val_loss: 0.5845 - val_accuracy: 0.6906
Epoch 8/20
32/32 [=

In [55]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.6061757206916809, 0.688249409198761]

In [56]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.61
accuracy: 68.82%
